# Сам декоратор

In [1]:
from functools import wraps
from collections import deque
import time


# Типы кэша.
LAST_CALLS = 0
POPULAR_CALLS = 1


# Выводит состояние переданных в параметры переменных до и после вызова функции.
def print_debug(active, *debug_args):
    if active:
        def decorator(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                print(f"DEBUG: {func.__name__}({', '.join(map(str, args))}, "
                      f"{', '.join(map(lambda x: f'{x}={kwargs[x]}', kwargs))})\n"
                      f"Before call function:\n" +
                      '\n'.join(map(lambda x: f"{x[0]}: {x[1]}", debug_args)) + '\n')
                ret = func(*args, **kwargs)
                print(f"After call function:\n" +
                      '\n'.join(map(lambda x: f"{x[0]}: {x[1]}", debug_args)) + '\n')
                return ret

            return wrapper

        return decorator
    else:
        def decorator(func):
            return func

        return decorator


def my_cache(cache_size=None,
             cache_type=LAST_CALLS,
             call_history_size=None,
             debug=False):
    # Задаем функцию, отвечающую за хранение и печать истории вызовов.
    if call_history_size is not None:
        call_history = deque()

        def print_call_history(func, args, kwargs):
            if len(call_history) >= call_history_size:
                call_history.pop()
            call_history.appendleft(
                f"{func.__name__}"
                f"({', '.join(map(str, args))}, "
                f"{', '.join(map(lambda x: f'{x}={kwargs[x]}', kwargs))})"
            )
            print('Call history:\n' + '\n'.join(call_history) + '\n')
    else:
        # Если хранить и печатать историю вызовов не надо, то создаем функцию, которая ничего не делает.
        def print_call_history(func, args, kwargs):
            pass

    # Если размер кэша не задан, то максимум, что мы должны делать - печатать историю вызовов.
    if cache_size is None:
        def decorator(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                print_call_history(func, args, kwargs)
                return func(*args, **kwargs)

            return wrapper

        return decorator

    # Если размер кэша отрицательный, то мы считаем его бесконечным(нелогично, но ладно), значит можно просто хранить
    # словарь, где ключ - это аргумент, а значение - результат работы функции при данном аргументе.
    if cache_size < 0:
        args_to_rets = dict()

        def decorator(func):
            @print_debug(debug, ("args_to_rets", args_to_rets))
            @wraps(func)
            def wrapper(*args, **kwargs):
                print_call_history(func, args, kwargs)
                # Преобразуем аргументы в неизменяемый тип, чтобы их можно было использовать в качестве ключа в словаре.
                args_as_key = args, tuple(sorted(kwargs))
                if args_as_key in args_to_rets:
                    return args_to_rets[args_as_key]
                ret = func(*args, **kwargs)
                args_to_rets[args_as_key] = ret
                return ret

            return wrapper

        return decorator

    # Если тип кэша - LAST_CALLS, то будем хранить словарь из аргументов в пару (возвраты, количество таких
    # аргументов в очереди) + очередь с историей вызовов.
    if cache_type == LAST_CALLS:
        queue_of_args = deque()
        args_to_rets = dict()

        def decorator(func):
            @print_debug(debug, ("args_to_rets", args_to_rets), ("queue_of_args", queue_of_args))
            @wraps(func)
            def wrapper(*args, **kwargs):
                print_call_history(func, args, kwargs)
                args_as_key = args, tuple(sorted(kwargs))

                # Если мы уже вызывали функцию с такими аргументами, то:
                if args_as_key in args_to_rets:
                    queue_of_args.appendleft(args_as_key)
                    # Увеличиваем счетчик вызовов с такими аргументами на 1.
                    args_to_rets[args_as_key][1] += 1
                    if len(queue_of_args) > cache_size:
                        to_del = queue_of_args.pop()
                        # Уменьшаем счетчик вызовов с такими аргументами на 1. Если счетчик стал равен нулю, 
                        # то удаляем такие аргументы из словаря. 
                        args_to_rets[to_del][1] -= 1
                        if args_to_rets[to_del][1] == 0:
                            args_to_rets.pop(to_del)
                    return args_to_rets[args_as_key][0]

                ret = func(*args, **kwargs)
                queue_of_args.appendleft(args_as_key)
                # Так как мы еще не вызывали функцию с такими аргументами, то счетчик вызовов с такими аргументами равен 1.
                args_to_rets[args_as_key] = [ret, 1]
                if len(queue_of_args) > cache_size:
                    to_del = queue_of_args.pop()
                    # Уменьшаем счетчик вызовов с такими аргументами на 1.
                    args_to_rets[to_del][1] -= 1
                    if args_to_rets[to_del][1] == 0:
                        args_to_rets.pop(to_del)
                return ret

            return wrapper

        return decorator

    # Если тип кэша - POPULAR_CALLS, то будем хранить словарь из аргументов в возвраты + словарь из аргументов в
    # количество вызовов функции с такими аргументами.
    if cache_type == POPULAR_CALLS:
        args_to_rets = dict()
        args_to_call_count = dict()

        def decorator(func):
            @print_debug(debug, ("args_to_rets", args_to_rets), ("args_to_call_count", args_to_call_count))
            @wraps(func)
            def wrapper(*args, **kwargs):
                print_call_history(func, args, kwargs)
                args_as_key = args, tuple(sorted(kwargs))
                # Если мы уже вызывали функцию от таких аргументов, то увеличиваем счетчик вызовов и возвращаем
                # результат.
                if args_as_key in args_to_rets:
                    args_to_call_count[args_as_key] += 1
                    return args_to_rets[args_as_key]
                # Если еще не вызывали, то ищем ключ с наименьшим количеством вызовов и удаляем его из обоих словарей.
                ret = func(*args, **kwargs)
                if len(args_to_rets) >= cache_size:
                    min_key = min(args_to_call_count,
                                  key=args_to_call_count.get)
                    args_to_rets.pop(min_key)
                    args_to_call_count.pop(min_key)
                args_to_rets[args_as_key] = ret
                args_to_call_count[args_as_key] = 1
                return ret

            return wrapper

        return decorator

Для тестов будем использовать декоратор, который печатает время выполнения функции

In [2]:
def clock(func):
    @wraps(func)
    def clocked(*args, **kwargs):
        t_start = time.time()
        res = func(*args, **kwargs)
        total_time = time.time() - t_start

        print(f"{func.__name__}("
              f"{', '.join(map(str, args))}, "
              f"{', '.join(map(lambda x: f'{x}={kwargs[x]}', kwargs))})"
              f" -> {res} executed in {total_time:.3f}")

        return res

    return clocked

Чтобы понимать насколько эффективен тот или иной вид кэша, будем считать с его помощью числа Фибоначчи.

Для начала попробуем посчитать определенное число Фибоначчи без кэша.

In [3]:
NUMBER_FOR_TESTS = 6

@clock
def calc_fib(n):
    time.sleep(1)

    if n < 2:
        return n

    return calc_fib(n - 1) + calc_fib(n - 2)

In [4]:
calc_fib(NUMBER_FOR_TESTS)

calc_fib(1, ) -> 1 executed in 1.002
calc_fib(0, ) -> 0 executed in 1.013
calc_fib(2, ) -> 1 executed in 3.021
calc_fib(1, ) -> 1 executed in 1.014
calc_fib(3, ) -> 2 executed in 5.049
calc_fib(1, ) -> 1 executed in 1.003
calc_fib(0, ) -> 0 executed in 1.006
calc_fib(2, ) -> 1 executed in 3.010
calc_fib(4, ) -> 3 executed in 9.060
calc_fib(1, ) -> 1 executed in 1.000
calc_fib(0, ) -> 0 executed in 1.003
calc_fib(2, ) -> 1 executed in 3.004
calc_fib(1, ) -> 1 executed in 1.003
calc_fib(3, ) -> 2 executed in 5.011
calc_fib(5, ) -> 5 executed in 15.083
calc_fib(1, ) -> 1 executed in 1.014
calc_fib(0, ) -> 0 executed in 1.016
calc_fib(2, ) -> 1 executed in 3.033
calc_fib(1, ) -> 1 executed in 1.012
calc_fib(3, ) -> 2 executed in 5.046
calc_fib(1, ) -> 1 executed in 1.007
calc_fib(0, ) -> 0 executed in 1.004
calc_fib(2, ) -> 1 executed in 3.013
calc_fib(4, ) -> 3 executed in 9.071
calc_fib(6, ) -> 8 executed in 25.163


8

На будущее запомним, что без кэша посчитать шестое число Фибоначчи занимает 25 секунд

# Ничего не делающий кэш

По заданию такое нужно было сделать

In [5]:
@clock
@my_cache(None)
def foo(x):
    time.sleep(1)
    return x

In [6]:
foo(1)

foo(1, ) -> 1 executed in 1.002


1

In [7]:
foo(1)

foo(1, ) -> 1 executed in 1.002


1

Видно, что время выполнения при одинаковых аргументах не меняется

# Кэш, хранящий историю вызовов

Тут ничего не кэшируется, просто хранится история вызовов

In [8]:
@clock
@my_cache(None, call_history_size=4)
def foo(x):
    time.sleep(1)
    return x

In [9]:
foo(1)

Call history:
foo(1, )

foo(1, ) -> 1 executed in 1.002


1

In [11]:
foo(2)

Call history:
foo(2, )
foo(2, )
foo(1, )

foo(2, ) -> 2 executed in 1.007


2

In [12]:
foo(1)

Call history:
foo(1, )
foo(2, )
foo(2, )
foo(1, )

foo(1, ) -> 1 executed in 1.013


1

In [13]:
foo(3)

Call history:
foo(3, )
foo(1, )
foo(2, )
foo(2, )

foo(3, ) -> 3 executed in 1.004


3

In [14]:
foo(4)

Call history:
foo(4, )
foo(3, )
foo(1, )
foo(2, )

foo(4, ) -> 4 executed in 1.006


4

Я реализовал историю в виде очереди. В предыдущей ячейке видно, что при новом вызове функции, информация о нем добавляется наверх, а информация о самом старом вызове удаляется

# Кэш, хранящий результаты всех вызовов

In [15]:
@clock
@my_cache(cache_size=-1, debug=True)
def foo(x):
    time.sleep(1)
    return x

In [16]:
foo(1)

DEBUG: foo(1, )
Before call function:
args_to_rets: {}

After call function:
args_to_rets: {((1,), ()): 1}

foo(1, ) -> 1 executed in 1.006


1

In [17]:
foo(1)

DEBUG: foo(1, )
Before call function:
args_to_rets: {((1,), ()): 1}

After call function:
args_to_rets: {((1,), ()): 1}

foo(1, ) -> 1 executed in 0.000


1

In [18]:
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}

foo(2, ) -> 2 executed in 1.007


2

In [19]:
foo(3)

DEBUG: foo(3, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2, ((3,), ()): 3}

foo(3, ) -> 3 executed in 1.017


3

In [20]:
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2, ((3,), ()): 3}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2, ((3,), ()): 3}

foo(2, ) -> 2 executed in 0.000


2

In [21]:
foo(3)

DEBUG: foo(3, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2, ((3,), ()): 3}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2, ((3,), ()): 3}

foo(3, ) -> 3 executed in 0.000


3

In [22]:
foo(1)

DEBUG: foo(1, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2, ((3,), ()): 3}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2, ((3,), ()): 3}

foo(1, ) -> 1 executed in 0.001


1

Проверим этот кэш на функции вычисления щестого числа Фибоначчи

In [23]:
@clock
@my_cache(cache_size=-1)
def calc_fib(n):
    time.sleep(1)

    if n < 2:
        return n

    return calc_fib(n - 1) + calc_fib(n - 2)

In [24]:
calc_fib(NUMBER_FOR_TESTS)

calc_fib(1, ) -> 1 executed in 1.000
calc_fib(0, ) -> 0 executed in 1.013
calc_fib(2, ) -> 1 executed in 3.013
calc_fib(1, ) -> 1 executed in 0.000
calc_fib(3, ) -> 2 executed in 4.014
calc_fib(2, ) -> 1 executed in 0.000
calc_fib(4, ) -> 3 executed in 5.023
calc_fib(3, ) -> 2 executed in 0.000
calc_fib(5, ) -> 5 executed in 6.028
calc_fib(4, ) -> 3 executed in 0.000
calc_fib(6, ) -> 8 executed in 7.031


8

Более чем в 3 раза быстрее, чем без кэша

# Кэш, хранящий результаты последних нескольких вызовов

In [25]:
@clock
@my_cache(cache_size=2, cache_type=LAST_CALLS, debug=True)
def foo(x):
    time.sleep(1)
    return x

In [26]:
foo(1)

DEBUG: foo(1, )
Before call function:
args_to_rets: {}
queue_of_args: deque([])

After call function:
args_to_rets: {((1,), ()): [1, 1]}
queue_of_args: deque([((1,), ())])

foo(1, ) -> 1 executed in 1.013


1

In [27]:
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): [1, 1]}
queue_of_args: deque([((1,), ())])

After call function:
args_to_rets: {((1,), ()): [1, 1], ((2,), ()): [2, 1]}
queue_of_args: deque([((2,), ()), ((1,), ())])

foo(2, ) -> 2 executed in 1.016


2

In [28]:
foo(1) # Так как вызывали раньше, то выполняется мгновенно

DEBUG: foo(1, )
Before call function:
args_to_rets: {((1,), ()): [1, 1], ((2,), ()): [2, 1]}
queue_of_args: deque([((2,), ()), ((1,), ())])

After call function:
args_to_rets: {((1,), ()): [1, 1], ((2,), ()): [2, 1]}
queue_of_args: deque([((1,), ()), ((2,), ())])

foo(1, ) -> 1 executed in 0.000


1

In [29]:
foo(3)

DEBUG: foo(3, )
Before call function:
args_to_rets: {((1,), ()): [1, 1], ((2,), ()): [2, 1]}
queue_of_args: deque([((1,), ()), ((2,), ())])

After call function:
args_to_rets: {((1,), ()): [1, 1], ((3,), ()): [3, 1]}
queue_of_args: deque([((3,), ()), ((1,), ())])

foo(3, ) -> 3 executed in 1.004


3

In [30]:
foo(2) # Среди предыдущих двух вызовов не было двойки, поэтому выполняется долго

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): [1, 1], ((3,), ()): [3, 1]}
queue_of_args: deque([((3,), ()), ((1,), ())])

After call function:
args_to_rets: {((3,), ()): [3, 1], ((2,), ()): [2, 1]}
queue_of_args: deque([((2,), ()), ((3,), ())])

foo(2, ) -> 2 executed in 1.011


2

Испытаем такой вариант на числах Фибоначчи

In [31]:
@clock
@my_cache(cache_size=3, cache_type=LAST_CALLS)
def calc_fib(n):
    time.sleep(1)

    if n < 2:
        return n

    return calc_fib(n - 1) + calc_fib(n - 2)

In [32]:
calc_fib(NUMBER_FOR_TESTS)

calc_fib(1, ) -> 1 executed in 1.005
calc_fib(0, ) -> 0 executed in 1.011
calc_fib(2, ) -> 1 executed in 3.016
calc_fib(1, ) -> 1 executed in 0.000
calc_fib(3, ) -> 2 executed in 4.016
calc_fib(2, ) -> 1 executed in 0.000
calc_fib(4, ) -> 3 executed in 5.031
calc_fib(3, ) -> 2 executed in 0.000
calc_fib(5, ) -> 5 executed in 6.044
calc_fib(4, ) -> 3 executed in 0.000
calc_fib(6, ) -> 8 executed in 7.055


8

Столько же, сколько у кэширования всех результатов

# Кэш, хранящий результаты популярных вызовов

In [40]:
@clock
@my_cache(cache_size=2, cache_type=POPULAR_CALLS, debug=True)
def foo(x):
    time.sleep(1)
    return x

In [41]:
foo(1)

DEBUG: foo(1, )
Before call function:
args_to_rets: {}
args_to_call_count: {}

After call function:
args_to_rets: {((1,), ()): 1}
args_to_call_count: {((1,), ()): 1}

foo(1, ) -> 1 executed in 1.001


1

In [42]:
foo(1)

DEBUG: foo(1, )
Before call function:
args_to_rets: {((1,), ()): 1}
args_to_call_count: {((1,), ()): 1}

After call function:
args_to_rets: {((1,), ()): 1}
args_to_call_count: {((1,), ()): 2}

foo(1, ) -> 1 executed in 0.000


1

In [43]:
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1}
args_to_call_count: {((1,), ()): 2}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 2, ((2,), ()): 1}

foo(2, ) -> 2 executed in 1.013


2

In [44]:
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 2, ((2,), ()): 1}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 2, ((2,), ()): 2}

foo(2, ) -> 2 executed in 0.000


2

In [45]:
foo(1) # Хоть и предыдущие 2 вызова это foo(2), но все равно foo(1) выполняется мгновенно

DEBUG: foo(1, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 2, ((2,), ()): 2}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 2}

foo(1, ) -> 1 executed in 0.000


1

In [46]:
foo(3) # Так как foo(1) вызывалась 3 раза, а foo(2) - 2, то foo(3) затрет foo(2)

DEBUG: foo(3, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 2}

After call function:
args_to_rets: {((1,), ()): 1, ((3,), ()): 3}
args_to_call_count: {((1,), ()): 3, ((3,), ()): 1}

foo(3, ) -> 3 executed in 1.000


3

In [47]:
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1, ((3,), ()): 3}
args_to_call_count: {((1,), ()): 3, ((3,), ()): 1}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 1}

foo(2, ) -> 2 executed in 1.011


2

In [48]:
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 1}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 2}

foo(2, ) -> 2 executed in 0.000


2

In [49]:
foo(2)
foo(2)
foo(2)

DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 2}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 3}

foo(2, ) -> 2 executed in 0.000
DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 3}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 4}

foo(2, ) -> 2 executed in 0.000
DEBUG: foo(2, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 4}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 5}

foo(2, ) -> 2 executed in 0.000


2

In [50]:
foo(1) # Даже после 3 вызовов foo(2), у нас все еще есть foo(1)

DEBUG: foo(1, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 3, ((2,), ()): 5}

After call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 4, ((2,), ()): 5}

foo(1, ) -> 1 executed in 0.000


1

In [51]:
foo(7) # Сейчас количество вызовов у двойки больше чем у единицы, поэтому семерка затрет единицу

DEBUG: foo(7, )
Before call function:
args_to_rets: {((1,), ()): 1, ((2,), ()): 2}
args_to_call_count: {((1,), ()): 4, ((2,), ()): 5}

After call function:
args_to_rets: {((2,), ()): 2, ((7,), ()): 7}
args_to_call_count: {((2,), ()): 5, ((7,), ()): 1}

foo(7, ) -> 7 executed in 1.010


7

Испытаем этот кэш на числах Фибоначчи

In [52]:
@clock
@my_cache(cache_size=4, cache_type=POPULAR_CALLS)
def calc_fib(n):
    time.sleep(1)

    if n < 2:
        return n

    return calc_fib(n - 1) + calc_fib(n - 2)

In [53]:
calc_fib(NUMBER_FOR_TESTS)

calc_fib(1, ) -> 1 executed in 1.000
calc_fib(0, ) -> 0 executed in 1.015
calc_fib(2, ) -> 1 executed in 3.015
calc_fib(1, ) -> 1 executed in 0.000
calc_fib(3, ) -> 2 executed in 4.015
calc_fib(2, ) -> 1 executed in 0.000
calc_fib(4, ) -> 3 executed in 5.015
calc_fib(3, ) -> 2 executed in 0.000
calc_fib(5, ) -> 5 executed in 6.029
calc_fib(3, ) -> 2 executed in 0.000
calc_fib(2, ) -> 1 executed in 0.000
calc_fib(4, ) -> 3 executed in 1.001
calc_fib(6, ) -> 8 executed in 8.042


8

На 1 секунду дольше чем у кэша, хранящего последние вызовы

# Теперь посмотрим какой тип кэша лучше

Храним популярные запросы

In [54]:
@my_cache(cache_size=4, cache_type=POPULAR_CALLS)
def calc_fib(n):
    time.sleep(1)

    if n < 2:
        return n

    return calc_fib(n - 1) + calc_fib(n - 2)

In [55]:
%%time

calc_fib(10)

Wall time: 37.1 s


55

Храним последние запросы

In [56]:
@my_cache(cache_size=4, cache_type=LAST_CALLS)
def calc_fib(n):
    time.sleep(1)

    if n < 2:
        return n

    return calc_fib(n - 1) + calc_fib(n - 2)

In [57]:
%%time

calc_fib(10)

Wall time: 11 s


55

Не храним кэш

In [58]:
def calc_fib(n):
    time.sleep(1)

    if n < 2:
        return n

    return calc_fib(n - 1) + calc_fib(n - 2)

In [59]:
%%time

calc_fib(10)

Wall time: 2min 57s


55

# Вывод

Получилось, что любой кэш сильно выгоднее обычной функции, но кэш, хранящий последние запросы, оказался лучше кэша, хранящего популярные запросы. Я думаю, это связано с тем, что функция на которой я тестировал кэши благоприятна первому кэшу, потому что в ней часто выполняются запросы, которые выполнялись совсем недавно. Например calc_fib(n) считает calc_fib(n-1), что в свою очередь считает calc_fib(n-2) и calc_fib(n-3) и посчитав их у нас в истории будет calc_fib(n-3), calc_fib(n-2), calc_fib(n-1), потом calc_fib(n) вызывает calc_fib(n-2) который у нас уже посчитан. Я думаю, что если не будет такой хорошей зависимости как в числах Фибоначчи, а, например, будет сайт, на котором пользователи будут выполнять какие-нибудь функции, тогда кэш с популярными запросами может хорошо себя проявить. С другой стороны, POPULAR_CALLS требует O(n) операций при работе, в отличии от LAST_CALLS, который требует для своей работы O(1) операций, но если мы не будем делать огромный кэш, то это вряд ли сильно повлияет, хотя тут уже зависит от функции, к которой мы хотим применить кэш